In [64]:
## fix auto reloading
import pandas
import os
import sys
from dotenv import load_dotenv
import json
import pandas as pd
from video_selection.google_trends import get_trends
from video_selection.search_by_keyword import write_keyword_and_id
from video_selection.filter import filter_sample_videos
from feature_extraction.metadata import metadata_extraction
from feature_extraction.metadata import youtube_authenticate
from feature_extraction.video import analyze_by_path
from feature_extraction.setup import *
from feature_extraction.audio import transcribe_gcs
from feature_extraction.NLP import *
from datetime import datetime
import pprint

# Load environment variables from the .env file
load_dotenv(override=True)

True

# Video Selection

In [7]:
# pipeline setup
keyword_path = './input/input_keyword_list.csv' # Modified for different disease, Input the keyword list
keyword_ref = 'sleep apnea causes' # use the first keyword for comparison in each iteration. 
num_of_result = 100

In [ ]:
# test for google trends
keyword_list = get_trends(keyword_path, keyword_ref, num_of_result)
print(keyword_list[:10])

In [ ]:
# test search for keyword
df_keyword = write_keyword_and_id(keyword_list, 5)
df_keyword.head(5)

In [ ]:
df_keyword

,keyword,id,rank,toKeep
304,excessive yawning,-MjaUhSOf94,5,False
367,difficulty falling asleep,PmM1IH-LaVQ,3,False
394,Obstructive Syndrome,krxcVBgerHM,5,False
480,pain in front center of brain,FSPzEzNBtMg,1,False
481,pain in front center of brain,HvVa7vUSRIo,2,False
482,pain in front center of brain,1JrdO-1Wf8g,3,False
483,pain in front center of brain,TYhgbE_84MA,4,False
484,pain in front center of brain,vPje60GgHXc,5,False


In [ ]:
# test filter and sample
credential_path = "oauth_credential.json"
# just for test
df_keyword = pd.read_csv('./temp/videoID_list.csv')
sampled_videos = filter_sample_videos(df_keyword, credential_path)
sampled_videos.head(10)

In [ ]:
# display the video_dictionary
df_video = pd.read_csv('temp/sampled_filtered_video_list.csv')
df_video.head()

,keyword,id,rank
0,ACROMEGALY,KLoaYPXlWHM,22
1,ACROMEGALY,OFl_bKVYsPo,9
2,ACROMEGALY,DAWYyBQJ8Jk,7
3,ACROMEGALY,AI-edvZx2GU,23
4,Anxious mood,YufQY6HqCW0,4


In [ ]:
df_video[df_video['id']== "KLoaYPXlWHM"].index[0]

0

# Extract Metadata

In [65]:
# pipeline setup
testID = "KLoaYPXlWHM"
test_keyword = "ACROMEGALY"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getcwd() + os.environ.get("SERVICE_ACCOUNT_PATH")
youtube = youtube_authenticate(os.environ.get("OAUTH_CREDENTIAL_PATH"))
local_video_path = "/Users/ZHANGRY/Codes/youtube-analytics-pipeline/temp/video"
local_audio_path = "/Users/ZHANGRY/Codes/youtube-analytics-pipeline/temp/audio"
bucket_name = "youtube_pipeline_testing"
gcs_bucket_path = os.path.join("gs://", bucket_name)
gcs_video_path = os.path.join(gcs_bucket_path, testID +".mp4")
gcs_audio_path = os.path.join(gcs_bucket_path, testID +".wav")

# download video to local 
download_video_by_id(testID, local_video_path)

# extract audio from video
extract_audios_from_videos(local_video_path, local_audio_path)

# upload video to GCP for video analysis
upload_files(bucket_name, local_video_path)

# upload audio to GCP for transcription analysis
upload_files(bucket_name, local_audio_path)

# get bucket information
# bucket_metadata(bucket_name)

Downloading: https://www.youtube.com/watch?v=KLoaYPXlWHM
Video successfully downloaded!
MoviePy - Writing audio in /Users/ZHANGRY/Codes/youtube-analytics-pipeline/temp/audio/KLoaYPXlWHM.wav


MoviePy - Done.
File /Users/ZHANGRY/Codes/youtube-analytics-pipeline/temp/video/KLoaYPXlWHM.mp4 uploaded to KLoaYPXlWHM.mp4.
File /Users/ZHANGRY/Codes/youtube-analytics-pipeline/temp/audio/KLoaYPXlWHM.wav uploaded to KLoaYPXlWHM.wav.


In [66]:
# raw metadata features
metadata = metadata_extraction(youtube, testID) # in json file

# video feature, 4 minutes for a video
video = analyze_by_path(gcs_video_path)

# transcription feature, 1 min 30 s for a video
audio = transcribe_gcs(gcs_audio_path)

# nlp feature
nlp = {}

# syntatic features of description and transcription
nlp['desc_words'],nlp['desc_uni'],nlp['desc_sen'],nlp['desc_act'], nlp['desc_sum'], nlp['desc_trans'], nlp['desc_ari'] = count_stats(metadata['description'])
nlp['tran_words'],nlp['tran_uni'],nlp['tran_sen'],nlp['tran_act'], nlp['tran_sum'], nlp['tran_trans'], nlp['tran_ari'] = count_stats(audio['transcription'])

# mer features of description and transcription
nlp['desc_mer'] = medical_entity_count(metadata['description'])
nlp['tran_mer'] = medical_entity_count(audio['transcription'])

# cosine similarity feature
nlp['cos_desc'] = calculate_cosine_similarity(test_keyword, metadata['description'])
nlp['cos_trans'] = calculate_cosine_similarity(test_keyword, audio['transcription'])
nlp['cos_title'] = calculate_cosine_similarity(test_keyword, metadata['title'])

# concat all features
merged_feature = {}
merged_feature.update(metadata)
merged_feature.update(video)
merged_feature.update(audio)
merged_feature.update(nlp)

# drop unnecessary columns
merged_feature.pop('description')
merged_feature.pop('transcription')
merged_feature.pop('title')

pprint.pprint(merged_feature)

extracting the id:  KLoaYPXlWHM
Retrieving accTag for:  https://www.youtube.com/watch?v=KLoaYPXlWHM
error in extracting accTag

Processing video for shot change annotations:

Finished processing.

Processing video for object annotations.

Finished processing.


Processing video for text detection.
Waiting for operation to complete...
Transcript: the voices of acromegaly a 360-degree look at that condition part to managing acromegaly A Day in the Life it's challenging to manage acromegaly patients should work with their Physicians to regularly Monitor and control their g h and igf-1 levels to help reduce the potential Health complications of acromegaly such as heart disease hypertension or diabetes there are also often psychosocial effects associated with this rare and life-threatening condition listen to the following people describe the day-to-day struggles of living with acromegaly the challenges faced and the adjustments they've had to make in order to cope with this condition
Confi

In [68]:
pprint.pprint(merged_feature)

{'accreditationTag': 0,
 'channel_subscribers': '33400',
 'cos_desc': 0.14675344736144216,
 'cos_title': 0.21951095080860836,
 'cos_trans': 0.10025657034850945,
 'desc_act': 30,
 'desc_ari': 8.581266233766232,
 'desc_mer': 2,
 'desc_sen': 7,
 'desc_sum': 0,
 'desc_trans': 0,
 'desc_uni': 34,
 'desc_words': 44,
 'duration': 259.0,
 'hasDescription': 1,
 'hasTags': 1,
 'hasThumbnail': 1,
 'id': 'KLoaYPXlWHM',
 'num_of_objects': 145,
 'num_of_shots': 9,
 'num_of_tags': 17,
 'publish_days': 2998,
 'text_confidence': 0.9719501769903934,
 'tran_act': 415,
 'tran_ari': 137.41669691470054,
 'tran_mer': 5,
 'tran_sen': 2,
 'tran_sum': 0,
 'tran_trans': 1,
 'tran_uni': 261,
 'tran_words': 551,
 'transcription confidence': 0.9354118853807449}


In [60]:
# concat all features
merged_feature = {}
merged_feature.update(metadata)
merged_feature.update(video)
merged_feature.update(audio)
merged_feature.update(nlp)

# drop unnecessary columns
merged_feature.pop('description')
merged_feature.pop('transcription')
merged_feature.pop('title')

pprint.pprint(merged_feature)

{'accreditationTag': 0,
 'channel_subscribers': '33400',
 'cos_desc': 0.14675344736144216,
 'cos_title': 0.21951095080860836,
 'cos_trans': 0.10025657034850945,
 'desc_act': 30,
 'desc_ari': 8.581266233766232,
 'desc_mer': 2,
 'desc_sen': 7,
 'desc_sum': 0,
 'desc_trans': 0,
 'desc_uni': 34,
 'desc_words': 44,
 'duration': 259.0,
 'hasDescription': 1,
 'hasTags': 1,
 'hasThumbnail': 1,
 'id': 'KLoaYPXlWHM.wav',
 'num_of_objects': 145,
 'num_of_shots': 9,
 'num_of_tags': 17,
 'publish_days': 2998,
 'text_confidence': 0.9719501787965948,
 'tran_act': 415,
 'tran_ari': 137.41669691470054,
 'tran_mer': 5,
 'tran_sen': 2,
 'tran_sum': 0,
 'tran_trans': 1,
 'tran_uni': 261,
 'tran_words': 551,
 'transcription confidence': 0.9354129061102867}


In [67]:
len(merged_feature)

32

In [15]:
!conda list -e > requirements.txt